In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls 
!cd sample_data

gdrive	sample_data  training_data.npy


In [0]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
REBUILD_DATA=True
class ForksVsSpoons():
    IMAGE_SIZE=50
    FORKS="/content/gdrive/My Drive/data/forky"
    SPOONS="/content/gdrive/My Drive/data/spoony"
    LABELS = {SPOONS: 0, FORKS: 1}
    training_data = []
    forkcount=0
    spooncount=0
    
    def make_training_data(self):
        for label in self.LABELS:
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMAGE_SIZE, self.IMAGE_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
                        
                        
                        if label == self.SPOONS:
                            self.spooncount += 1
                        elif label == self.FORKS:
                            self.forkcount += 1
                        
                    except Exception as e:
                        print(label, f, str(e))
                    
        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print('Forks:',self.forkcount)
        print('Spoons:',self.spooncount)
        
if REBUILD_DATA:
    forksvsspoons=ForksVsSpoons()
    forksvsspoons.make_training_data()
    
training_data = np.load("/content/gdrive/My Drive/training_data.npy", allow_pickle=True)
print(len(training_data))

100%|██████████| 1145/1145 [00:02<00:00, 436.77it/s]


Forks: 1145
Spoons: 1154
2299


In [0]:
class Net(nn.Module):
    def __init__(self):
        super().__init__() 
        self.conv1 = nn.Conv2d(1, 32, 5) 
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, 2)

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        #print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) 
        return F.softmax(x, dim=1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = Net().to(device)
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [0]:
optimizer = optim.Adam(net.parameters(), lr=0.01)
loss_function = nn.MSELoss()

In [0]:
X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

In [0]:
VAL_PCT = 0.1  
val_size = int(len(X)*VAL_PCT)
print(val_size)

229


In [0]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

In [0]:
print(len(train_X), len(test_X))

2070 229


In [0]:
BATCH_SIZE = 100
EPOCHS = 50

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
        net.zero_grad()

        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50).to(device)
        batch_y = train_y[i:i+BATCH_SIZE].to(device)

        

        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch}. Loss: {loss}")

  0%|          | 0/21 [00:00<?, ?it/s]

Epoch: 0. Loss: 0.4714285731315613
Epoch: 1. Loss: 0.4714285731315613


100%|██████████| 21/21 [00:00<00:00, 107.32it/s]


Epoch: 2. Loss: 0.4714285731315613
Epoch: 3. Loss: 0.4714285731315613


  0%|          | 0/21 [00:00<?, ?it/s]

...

Epoch: 44. Loss: 0.4714285731315613
Epoch: 45. Loss: 0.4714285731315613


  0%|          | 0/21 [00:00<?, ?it/s]

Epoch: 46. Loss: 0.4714285731315613
Epoch: 47. Loss: 0.4714285731315613


100%|██████████| 21/21 [00:00<00:00, 109.22it/s]

Epoch: 48. Loss: 0.4714285731315613
Epoch: 49. Loss: 0.4714285731315613


In [0]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50).to(device))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

100%|██████████| 229/229 [00:00<00:00, 1399.08it/s]

Accuracy:  0.489
